In [136]:
import numpy as np
import numpy.linalg as LA
from math import *
N = 0
accuracy = [1e-3, 1e-5]
startPoint = np.array([-1, 1])
koeffs = [1]

# def A(x):
#   return np.array([[200*x[0]*x[0] - 400*x[1] + 2, -200*x[0]], [-200*x[0], 200]], dtype=np.int64)
A = np.array([[2, 0], [0, 200]], dtype=np.int64)

In [137]:
import pandas as pd
results = pd.DataFrame(columns=['title', 'koeff', *[f'EPS={i}' for i in accuracy]])

In [138]:
results

,title,koeff,EPS=0.001,EPS=1e-05


In [139]:
def func(x, a):
  global N
  N += 1
  return 100 * (x[0] * x[0] - x[1]) ** 2 + (x[0] -1) ** 2

def fD0(x, a):
  global N
  N += 1
  return 2 * (200 * x[0] * (x[0]*x[0] - x[1]) + x[0] - 1)

def fD1(x, a):
  global N
  N += 1
  return -200 * (x[0] * x[0] - x[1])

def gradient(x, a):
  return np.array([fD0(x, a), fD1(x, a)])

In [140]:
def gradientDescent(startPoint, epsilon, koeff, A):

    point = np.array(startPoint)
    grad = gradient(point, koeff)
    

    while (LA.norm(grad) >= epsilon):
        alpha_k = - np.dot(grad, -grad)/np.dot(A @ -grad, -grad)
        point = point - alpha_k * grad

        grad = gradient(point, koeff)

    return (point, func(point, koeff))


In [141]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        gradientDescent(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"gradientDescent", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1]}).to_frame().T], ignore_index=True)


In [142]:
print(gradientDescent(startPoint, 1, 1e-3, A))
print(N)

(array([1., 1.]), 0.0)
5


In [143]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5


In [144]:
def conjugateGradientMethod(startPoint, epsilon, koeff, A):
    point = np.array(startPoint, dtype=np.int64)
    grad = gradient(point, koeff)
    p_k = -grad
    k = 0

    while (LA.norm(grad) >= epsilon):
        alpha_k = - np.dot(grad, p_k)/np.dot(A @ p_k, p_k)
        point = point + alpha_k * p_k
        Ngrad = gradient(point, koeff)

        if (k + 1 == point.size):
            betta_k=0
        else:
            betta_k = LA.norm(Ngrad)**2/LA.norm(grad)**2
        p_k = -Ngrad + np.dot(betta_k, p_k)

        grad = Ngrad

    return (point, func(point, koeff))


In [145]:
conjugateGradientMethod(startPoint, 1e-3, 1, A)

(array([1., 1.]), 0.0)

In [146]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        conjugateGradientMethod(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"conjugateGradientMethod", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1]}).to_frame().T], ignore_index=True)

In [147]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5
1,conjugateGradientMethod,1,15,5


In [148]:
def newtonsMethod(startPoint, epsilon, koeff, A):
    point = np.array(startPoint, dtype=np.int64)
    grad = gradient(point, koeff)
    invA = LA.inv(A)

    while (LA.norm(grad) >= epsilon):
        point = point - (invA @ grad)
        grad = gradient(point, koeff)
    return (point, func(point, koeff))

In [149]:
for koeff in koeffs:
    N_EPS = []

    for eps in accuracy:
        newtonsMethod(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"newtonsMethod", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1]}).to_frame().T], ignore_index=True)

In [150]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5
1,conjugateGradientMethod,1,15,5
2,newtonsMethod,1,5,5


In [151]:
def rSimplex(startPoint, ribLength, epsilon, koeff):

    simplex = buildSimplexC(startPoint, ribLength, lambda x: func(x, koeff))
    simplex = sort(simplex)
    k = 0
    while ribLength > epsilon or k < simplex.shape[0]:
        if k == simplex.shape[0]:
            k = 0
            ribLength = ribLength * 0.5
            simplex = buildSimplexD(simplex[-1], ribLength, lambda x: func(x, koeff))
            continue
        newApex = reflect(simplex, k, lambda x: func(x, koeff))
        if (newApex[-1] >= simplex[k][-1]):
            k += 1
            continue
        else:
            simplex[k] = newApex
            simplex = sort(simplex)
            k = 0
    return simplex[-1]


def sort(simplex):
    return simplex[simplex[:, -1].argsort()][::-1]


def reflect(simplex, k, func):
    coords = np.vstack((simplex[k+1:, :-1], simplex[:k, :-1]))
    center = np.sum(coords, axis=0)

    newCoord = 2 * center / coords.shape[0] - simplex[k, :-1]
    newValue = func(newCoord)

    newApex = np.hstack((newCoord, newValue))
    return newApex


def buildSimplexD(minApex, ribLength, func):
    numberOfDimensions = minApex[:-1].shape[0]
    apexPoints = np.array(minApex)

    for i in range(2, numberOfDimensions + 2):
        coords = np.array([])
        for j in range(1, numberOfDimensions + 1):
            if (i == j + 1):
                newCoord = minApex[j - 1] + (sqrt(numberOfDimensions + 1) - 1)/(numberOfDimensions * sqrt(2)) * ribLength
                coords = np.append(coords, newCoord)
            else:
                newCoord = minApex[j - 1] + (sqrt(numberOfDimensions + 1) + numberOfDimensions - 1)/(
                    numberOfDimensions * sqrt(2)) * ribLength
                coords = np.append(coords, newCoord)
        coords = np.append(coords, (func(coords)))
        apexPoints = np.vstack((apexPoints, coords))
    # print('New simplex is: \n', apexPoints)
    return apexPoints


def buildSimplexC(centerPoint, ribLength, func):
    '''
    Для реализации выбран второй способ построения симплекса через центральную точку

    Parameters
    ----------
    centerPoint (np.array): центр симплекса
    ribLength (int): длинна ребра симплекса
    func(vector): float

    Returns
    -------
    np.array(x, y, z, etc, val)
    '''
    numberOfDimensions = centerPoint.shape[0]
    apexPoints = None
    for i in range(1, numberOfDimensions + 2):
        # i это номер вершины симплекса
        coords = np.array([])
        for j in range(1, numberOfDimensions + 1):
            # j это номер координаты симплекса (x=1, y=2, z=3, ...)
            if j < i-1:
                newCoord = centerPoint[j - 1]
                coords = np.append(coords, newCoord)
            elif j == i-1:
                newCoord = centerPoint[j - 1] + sqrt(j / (2 * j + 2)) * ribLength
                coords = np.append(coords, newCoord)
            elif j > i-1:
                newCoord = centerPoint[j - 1] - sqrt(1 / (j*(2*j + 2))) * ribLength
                coords = np.append(coords, newCoord)
        coords = np.append(coords, (func(coords)))
        apexPoints = np.array(coords) if apexPoints is None else np.vstack(
            (apexPoints, coords))
    return apexPoints


In [152]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        rSimplex(startPoint, 0.5, eps,  koeff)
        N_EPS.append(N)
        N = 0
    results = pd.concat(
        [results, pd.Series(
            {"title": "rSimplex", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1]}).to_frame().T],
        ignore_index=True)


In [153]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5
1,conjugateGradientMethod,1,15,5
2,newtonsMethod,1,5,5
3,rSimplex,1,1123,12361


In [154]:
a = np.array([1, 2])
b = np.array([3, 4])
LA.norm(a-b)

2.8284271247461903

In [155]:
def coordinateDescent(startPoint, epsilon, koeff):
  newPoint = oldPoint = startPoint
  newValue = oldValue = func(startPoint, koeff)
  dimensionNumber = startPoint.shape[0]

  j = 1
  while (True):
    oldPoint = newPoint
    oldValue = newValue

    while j <= dimensionNumber:
      newPoint = bitwiseSearch(newPoint, j, oldValue, 0.5, lambda x: func(x, koeff), epsilon)
      j += 1

    newValue = func(newPoint, koeff)
    if isSolved(oldPoint, newPoint, oldValue, newValue, epsilon, epsilon ):
      break
    j = 1
  return (newPoint, newValue)

def bitwiseSearch(point, j, value, step, func, eps):
  oldValue = value
  newPoint = point
  directionVector = np.array([0] * point.shape[0])
  directionVector[j - 1] = 1
  # Micro-optimization
  if func(newPoint + step * directionVector) > func(newPoint - step * directionVector):
    step *= -1
  #
  while abs(step) > eps:
    newPoint = newPoint + step * directionVector
    newValue = func(newPoint)
    if (newValue > oldValue):
      step = -step / 4
    oldValue = newValue
  return newPoint

def isSolved(oldPoint, newPoint, oldValue, newValue, eps1, eps2):
  return LA.norm(oldPoint - newPoint) < eps1 or abs(oldValue - newValue) < eps2

In [156]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        coordinateDescent(startPoint, eps,  koeff)
        N_EPS.append(N)
        N = 0
    results = pd.concat(
        [results, pd.Series(
            {"title": "coordinateDescent", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1]}).to_frame().T],
        ignore_index=True)

In [157]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5
1,conjugateGradientMethod,1,15,5
2,newtonsMethod,1,5,5
3,rSimplex,1,1123,12361
4,coordinateDescent,1,4366,56522


In [158]:
def HookeJeeves(startPoint, eps,  koeff):
  incrementVector = np.array([1] * startPoint.shape[0])
  oldPoint = startPoint
  decrementKoeff = 2

  def isSolved(vector, eps):
    return LA.norm(vector) < eps

    
  while True:
    prePoint = preResearch(oldPoint, incrementVector, lambda x: func(x, koeff))
    if (oldPoint == prePoint).all():

      if isSolved(incrementVector, eps):
        break

      incrementVector = incrementVector / decrementKoeff
      continue
    oldPoint = oldPoint + (prePoint - oldPoint)
  return (oldPoint, func(oldPoint, koeff))



def preResearch(point, incrementVector, func):
  j = 1
  dimensionNumber = point.shape[0]
  baseVector = np.array([0] * dimensionNumber)
  oldPoint = point
  oldValue = func(oldPoint)

  while j <= dimensionNumber:
    baseVector[j - 1] = 1
    newPoint = point - incrementVector[j-1]*baseVector
    newValue = func(newPoint)
    if oldValue > newValue:
      oldPoint = newPoint
      oldValue = newValue
      baseVector[j - 1] = 0
      j += 1
      continue
    
    newPoint = point + incrementVector[j-1]*baseVector
    newValue = func(newPoint)
    if oldValue > newValue:
      oldPoint = newPoint
      oldValue = newValue
      baseVector[j - 1] = 0
      j += 1
      continue
    baseVector[j - 1] = 0
    j += 1
    
  return oldPoint

In [159]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        HookeJeeves(startPoint, eps,  koeff)
        N_EPS.append(N)
        N = 0
    results = pd.concat(
        [results, pd.Series(
            {"title": "HookeJeeves", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1]}).to_frame().T],
        ignore_index=True)

In [160]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5
1,conjugateGradientMethod,1,15,5
2,newtonsMethod,1,5,5
3,rSimplex,1,1123,12361
4,coordinateDescent,1,4366,56522
5,HookeJeeves,1,7803,32988


In [161]:
def randomSearch(startPoint, eps,  koeff):
  step = 2
  M = startPoint.shape[0] * 10
  newPoint = oldPoint = startPoint
  oldValue = func(startPoint, koeff)
  gamma = 2
  while step > eps:
    counter = 1
    while counter < M:
      randomVector = np.array([np.random.uniform(low=-1, high=1), np.random.uniform(low=-1, high=1)])
      newPoint = oldPoint + randomVector * step / LA.norm(randomVector)
      newValue = func(newPoint, koeff)
      if newValue >= oldValue:
        counter += 1
        continue
      while (newValue < oldValue):
        newPoint = newPoint + randomVector * step / LA.norm(randomVector)
        newValue = func(newPoint, koeff)
        oldValue = newValue
        oldPoint = newPoint
      break
    step /= gamma
  return (oldPoint, oldValue)

In [162]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        randomSearch(startPoint, eps,  koeff)
        N_EPS.append(N)
        N = 0
        print('Koeff:', koeff, 'Eps:', eps)
    results = pd.concat(
        [results, pd.Series(
            {"title": "randomSearch", "koeff": koeff, "EPS=0.001": N_EPS[0],
             "EPS=1e-05": N_EPS[1]}).to_frame().T],
        ignore_index=True)

Koeff: 1 Eps: 0.001
Koeff: 1 Eps: 1e-05


In [163]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5
1,conjugateGradientMethod,1,15,5
2,newtonsMethod,1,5,5
3,rSimplex,1,1123,12361
4,coordinateDescent,1,4366,56522
5,HookeJeeves,1,7803,32988
6,randomSearch,1,29,81


In [164]:
import pandas as pd
results = pd.DataFrame(columns=['title', 'koeff', *[f'EPS={i}' for i in accuracy]])

In [165]:
def gradientDescent(startPoint, epsilon, koeff, A):

    point = np.array(startPoint)
    grad = gradient(point, koeff)
    

    while (LA.norm(point - np.array([1, 1])) >= epsilon):
        alpha_k = - np.dot(grad, -grad)/np.dot(A @ -grad, -grad)
        point = point - alpha_k * grad

        grad = gradient(point, koeff)

    return (point, func(point, koeff))

def conjugateGradientMethod(startPoint, epsilon, koeff, A):
    point = np.array(startPoint, dtype=np.int64)
    grad = gradient(point, koeff)
    p_k = -grad
    k = 0

    while (LA.norm(point - np.array([1, 1])) >= epsilon):
        alpha_k = - np.dot(grad, p_k)/np.dot(A @ p_k, p_k)
        point = point + alpha_k * p_k
        Ngrad = gradient(point, koeff)

        if (k + 1 == point.size):
            betta_k=0
        else:
            betta_k = LA.norm(Ngrad)**2/LA.norm(grad)**2
        p_k = -Ngrad + np.dot(betta_k, p_k)

        grad = Ngrad

    return (point, func(point, koeff))

def newtonsMethod(startPoint, epsilon, koeff, A):
    point = np.array(startPoint, dtype=np.int64)
    grad = gradient(point, koeff)
    invA = LA.inv(A)

    while (LA.norm(point - np.array([1, 1])) >= epsilon):
        point = point - (invA @ grad)
        grad = gradient(point, koeff)
    return (point, func(point, koeff))


In [166]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        gradientDescent(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"gradientDescent", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1]}).to_frame().T], ignore_index=True)


In [167]:
for koeff in koeffs:
    N_EPS = []
    for eps in accuracy:
        conjugateGradientMethod(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"conjugateGradientMethod", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1]}).to_frame().T], ignore_index=True)

In [168]:
for koeff in koeffs:
    N_EPS = []

    for eps in accuracy:
        newtonsMethod(startPoint, eps, koeff, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"newtonsMethod", "koeff": koeff, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1]}).to_frame().T], ignore_index=True)

In [169]:
results

,title,koeff,EPS=0.001,EPS=1e-05
0,gradientDescent,1,5,5
1,conjugateGradientMethod,1,5,5
2,newtonsMethod,1,5,5


In [170]:
def coordinateDescent(startPoint, epsilon, epsilon2, koeff):
  newPoint = oldPoint = startPoint
  newValue = oldValue = func(startPoint, koeff)
  dimensionNumber = startPoint.shape[0]

  j = 1
  while (True):
    oldPoint = newPoint
    oldValue = newValue

    while j <= dimensionNumber:
      newPoint = bitwiseSearch(newPoint, j, oldValue, 0.5, lambda x: func(x, koeff), epsilon2)
      j += 1

    newValue = func(newPoint, koeff)
    if isSolved(oldPoint, newPoint, oldValue, newValue, epsilon, epsilon ):
      break
    j = 1
  return (newPoint, newValue)

def bitwiseSearch(point, j, value, step, func, eps):
  oldValue = value
  newPoint = point
  directionVector = np.array([0] * point.shape[0])
  directionVector[j - 1] = 1
  # Micro-optimization
  if func(newPoint + step * directionVector) > func(newPoint - step * directionVector):
    step *= -1
  #
  while abs(step) > eps:
    newPoint = newPoint + step * directionVector
    newValue = func(newPoint)
    if (newValue > oldValue):
      step = -step / 4
    oldValue = newValue
  return newPoint

def isSolved(oldPoint, newPoint, oldValue, newValue, eps1, eps2):
  return LA.norm(oldPoint - newPoint) < eps1 or abs(oldValue - newValue) < eps2

In [171]:
accuracy = [1e-3, 1e-5]
startPoint = np.array([-1, 1])
koeffs = [1]
innerAcc = [1e-4, 1e-6, 1e-8]

In [172]:
results2 = pd.DataFrame(columns=['title', 'koeff', 'EPS1', 'EPS2', 'N'])
for eps1 in accuracy:
  for eps2 in innerAcc:
    N = 0
    coordinateDescent(startPoint, eps1, eps2, 1)
    results2 = pd.concat([results2, pd.Series({"title":"coordinateDescent", "koeff": koeff, "EPS1":eps1,"EPS2":eps2, "N": N}).to_frame().T], ignore_index=True)
    

In [173]:
results2

,title,koeff,EPS1,EPS2,N
0,coordinateDescent,1,0.001,0.0001,10057
1,coordinateDescent,1,0.001,0.000001,15839
2,coordinateDescent,1,0.001,0.0,20424
3,coordinateDescent,1,0.00001,0.0001,53669
4,coordinateDescent,1,0.00001,0.000001,73550
5,coordinateDescent,1,0.00001,0.0,96256


In [174]:
print(gradientDescent(startPoint, 1e-6, 1, A))
print(conjugateGradientMethod(startPoint, 1e-6, 1, A))
print(newtonsMethod(startPoint, 1e-6, 1, A))
print(rSimplex(startPoint, 2, 1e-6, 1))
print(coordinateDescent(startPoint, 1e-6, 1e-6, koeff))
print(HookeJeeves(startPoint, 1e-6,  1))
print(randomSearch(startPoint, 1e-6,  1))


(array([1., 1.]), 0.0)
(array([1., 1.]), 0.0)
(array([1., 1.]), 0.0)
[1.00035101e+00 1.00070348e+00 1.23387628e-07]
(array([0.98623657, 0.97266006]), 0.00018943257390172455)
(array([0.99990273, 0.99980545]), 9.462391830914517e-09)
(array([0.32123915, 0.09865937]), 0.4627731107003272)


In [190]:
print(randomSearch(startPoint, 1e-6,  1))

(array([0.89944459, 0.80815941]), 0.01018214548002577)
